In [878]:
import pandas as pd

In [879]:
# Read CSV and convert some columns to apprropriate data type
df = pd.read_csv('trade.csv')
df.exports = pd.to_numeric(df.exports).astype(float)
df.imports = pd.to_numeric(df.imports).astype(float)

In [880]:
df['botg'] = df.apply(lambda row: row.exports - row.imports, axis=1)
df['total_trade'] = df.apply(lambda row: row.exports + row.imports, axis=1)

In [881]:
df_all = df.copy().set_index('year').sum(numeric_only=True)
df_all

exports        1468245.28
imports        1917833.93
botg           -449588.65
total_trade    3386079.21
dtype: float64

In [882]:
df['imports_growth_rate'] = df['imports'].pct_change()
df['exports_growth_rate'] = df['exports'].pct_change()
df['botg_growth_rate'] = df['botg'].pct_change()
df['total_trade_growth_rate'] = df['total_trade'].pct_change()

In [883]:
df.head()

,year,month,exports,imports,botg,total_trade,imports_growth_rate,exports_growth_rate,botg_growth_rate,total_trade_growth_rate
0,1991,Jan,635.57,950.41,-314.84,1585.98,NaN,NaN,NaN,NaN
1,1991,Feb,662.65,1094.81,-432.16,1757.46,0.151934,0.042607,0.372634,0.108122
2,1991,Mar,742.13,1076.87,-334.74,1819.00,-0.016386,0.119943,-0.225426,0.035016
3,1991,Apr,709.70,896.52,-186.82,1606.22,-0.167476,-0.043699,-0.441895,-0.116976
4,1991,May,688.51,983.15,-294.64,1671.66,0.096629,-0.029858,0.577133,0.040742


In [884]:
# Aggregate yearly
column_rates = ["imports_growth_rate", "exports_growth_rate", "botg_growth_rate", "total_trade_growth_rate"]
column_rates_src = ["imports", "exports", "botg", "total_trade"]

df_year = df.copy()
df_year.set_index('year')
df_year = df_year.groupby('year')
df_year = df_year.sum(numeric_only=True)

for i, column_rate in enumerate(column_rates):
    column_rate_src = column_rates_src[i]
    df_year[column_rate] = df_year[column_rate_src].pct_change()

In [885]:
df_year.loc[1991]

exports                     8839.51
imports                    12051.36
botg                       -3211.85
total_trade                20890.87
imports_growth_rate             NaN
exports_growth_rate             NaN
botg_growth_rate                NaN
total_trade_growth_rate         NaN
Name: 1991, dtype: float64

In [886]:
df_month_year = df.pivot(index="year", columns="month")

In [887]:
df_month_year.head()

exports                                                                 \
month      Apr      Aug      Dec      Feb      Jan      Jul      Jun      Mar   
year                                                                            
1991    709.70   741.20   826.13   662.65   635.57   793.50   767.70   742.13   
1992    666.45   913.09   912.80   714.90   662.37   851.13   835.57   898.46   
1993    861.73   994.50  1004.05   842.71   790.44   998.17   914.44   842.46   
1994    965.79  1178.01  1308.55   927.01   951.62  1217.56  1125.27  1057.67   
1995   1377.42  1591.45  1621.01  1263.77  1160.75  1592.69  1521.69  1298.58   

                         ... total_trade_growth_rate                      \
month      May      Nov  ...                     Dec       Feb       Jan   
year                     ...                                               
1991    688.51   751.08  ...               -0.020466  0.108122       NaN   
1992    811.71   824.14  ...                0.076155  0.027852 -0.067454   
1993    932.14  1163.36  ...               -0.094540  0.025516 -0.117764   
1994   1083.86  1128.46  ...                0.071083 -0.125330  0.009759   
1995   1400.58  1422.63  ...                0.092525 -0.045310 -0.057891   

                                                                             
month       Jul       Jun       Mar       May       Nov       Oct       Sep  
year                                                                         
1991   0.066592  0.020309  0.035016  0.040742  0.045555  0.015454  0.025933  
1992   0.048334 -0.036777  0.180759  0.072314 -0.051101 -0.008099 -0.044649  
1993   0.134283 -0.059713  0.103063  0.011286  0.067070  0.032246  0.030600  
1994  -0.057193  0.094305  0.167457  0.055599 -0.061800 -0.006731  0.103759  
1995  -0.021341  0.086378  0.194974 -0.017402 -0.027044 -0.023564  0.045058  

[5 rows x 96 columns]

In [888]:
# Compare values of the same year (eg: Exports of February and March in 1991)
df_1991 = df_month_year.loc[1991]['imports']

In [889]:
df.head()

,year,month,exports,imports,botg,total_trade,imports_growth_rate,exports_growth_rate,botg_growth_rate,total_trade_growth_rate
0,1991,Jan,635.57,950.41,-314.84,1585.98,NaN,NaN,NaN,NaN
1,1991,Feb,662.65,1094.81,-432.16,1757.46,0.151934,0.042607,0.372634,0.108122
2,1991,Mar,742.13,1076.87,-334.74,1819.00,-0.016386,0.119943,-0.225426,0.035016
3,1991,Apr,709.70,896.52,-186.82,1606.22,-0.167476,-0.043699,-0.441895,-0.116976
4,1991,May,688.51,983.15,-294.64,1671.66,0.096629,-0.029858,0.577133,0.040742


In [890]:
all_exports = pd.DataFrame(df["exports"]).rename(columns={"exports": "values"})
all_exports['type'] = 'exports'
all_exports

,values,type
0,635.57,exports
1,662.65,exports
2,742.13,exports
3,709.70,exports
4,688.51,exports
...,...,...
391,6729.64,exports
392,6771.93,exports
393,6518.43,exports
394,6231.07,exports


In [891]:
all_imports = pd.DataFrame(df["imports"]).rename(columns={"imports": "values"})
all_imports['type'] = 'imports'
all_imports

,values,type
0,950.41,imports
1,1094.81,imports
2,1076.87,imports
3,896.52,imports
4,983.15,imports
...,...,...
391,10834.74,imports
392,10320.24,imports
393,10759.41,imports
394,11000.53,imports


In [892]:
all = pd.concat([all_exports, all_imports])
all

,values,type
0,635.57,exports
1,662.65,exports
2,742.13,exports
3,709.70,exports
4,688.51,exports
...,...,...
391,10834.74,imports
392,10320.24,imports
393,10759.41,imports
394,11000.53,imports


In [893]:
import plotly.express as px

df_tips = px.data.tips()
df_tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [894]:
px.pie(data_frame=all, values='values', names='type')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': 'type=%{label}<br>value=%{value}<extra></extra>',
              'labels': array(['exports', 'exports', 'exports', ..., 'imports', 'imports', 'imports'],
                              dtype=object),
              'legendgroup': '',
              'name': '',
              'showlegend': True,
              'type': 'pie',
              'values': array([  635.57,   662.65,   742.13, ..., 10759.41, 11000.53,  9962.5 ])}],
    'layout': {'legend': {'tracegroupgap': 0}, 'margin': {'t': 60}, 'template': '...'}
})